In [3]:
import os
import zipfile as zf

# Path to the repaired ZIP file
zip_file_path = "PartB_DFU_dataset - Copy.zip"
extract_path = "DFU_dataset"

if os.path.exists(zip_file_path):
    try:
        with zf.ZipFile(zip_file_path, 'r') as files:
            files.extractall(extract_path)
        print(f"Extraction completed successfully to '{extract_path}'")
    except zf.BadZipFile:
        print("Error: The ZIP file is corrupted.")
    except OSError as e:
        print(f"OS error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
else:
    print(f"Error: The file '{zip_file_path}' does not exist.")


Extraction completed successfully to 'DFU_dataset'


In [4]:
import numpy as np # linear algebra
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
from PIL import Image
# Define the root directory where your image folders are located
root_directory = "DFU_dataset/PartB_DFU_dataset - Copy"

# Initialize lists to store image paths and corresponding class labels for both datasets
image_paths_ischaemia = []
categories_ischaemia = []
image_paths_infection = []
categories_infection = []

# Iterate over each class and its subdirectories
for class_name in ["Infection", "Ischaemia"]:
    for augmentation_type in ["Aug-Negative", "Aug-Positive"]:
        folder_path = os.path.join(root_directory, class_name, augmentation_type)
        category = f"{class_name.lower()}{'pov' if 'Positive' in augmentation_type else 'neg'}"
        
        # Iterate over image files in the current directory
        for file_name in os.listdir(folder_path):
            if file_name.endswith(".jpg"):  # Assuming images are jpg format
                image_path = os.path.join(folder_path, file_name)
                if class_name == "Ischaemia":
                    image_paths_ischaemia.append(image_path)
                    categories_ischaemia.append("ischemia" if "Positive" in augmentation_type else "non-ischemia")
                elif class_name == "Infection":
                    image_paths_infection.append(image_path)
                    categories_infection.append("infection" if "Positive" in augmentation_type else "non-infection")

# Create DataFrames for each dataset
df_ischaemia = pd.DataFrame({"category": categories_ischaemia, "image_path": image_paths_ischaemia})
df_infection = pd.DataFrame({"category": categories_infection, "image_path": image_paths_infection})

# Label encoding for Ischaemia dataset
label_encoder_ischaemia = LabelEncoder()
df_ischaemia['Class_Label'] = label_encoder_ischaemia.fit_transform(df_ischaemia['category'])
print("Ischaemia Class Mapping:")
for class_label, numerical_label in zip(df_ischaemia['category'].unique(), df_ischaemia['Class_Label'].unique()):
    print(f"{class_label}: {numerical_label}")

# Label encoding for Infection dataset
label_encoder_infection = LabelEncoder()
df_infection['Class_Label'] = label_encoder_infection.fit_transform(df_infection['category'])
print("Infection Class Mapping:")
for class_label, numerical_label in zip(df_infection['category'].unique(), df_infection['Class_Label'].unique()):
    print(f"{class_label}: {numerical_label}")

# Shuffle both DataFrames
df_ischaemia = df_ischaemia.sample(frac=1).reset_index(drop=True)
df_infection = df_infection.sample(frac=1).reset_index(drop=True)

# Helper function to load and process images
def load_images(df):
    images = []
    target_labels = []   
    for index, row in df.iterrows():
        image = Image.open(row['image_path'])
        image_array = np.array(image.resize((224, 224)))  # Resize image to fit MobileNet input size
        images.append(image_array)
        target_labels.append(row['Class_Label'])
    return np.array(images), np.array(target_labels)

# Load images for both datasets
images_ischaemia, target_labels_ischaemia = load_images(df_ischaemia)
images_infection, target_labels_infection = load_images(df_infection)

print("Shape of Ischaemia images array:", images_ischaemia.shape)
print("Shape of Ischaemia target labels array:", target_labels_ischaemia.shape)
print("Shape of Infection images array:", images_infection.shape)
print("Shape of Infection target labels array:", target_labels_infection.shape)

# Split the Ischaemia dataset
X_train_ischaemia, X_test_ischaemia, y_train_ischaemia, y_test_ischaemia = train_test_split(
    images_ischaemia, target_labels_ischaemia, test_size=0.3, random_state=42)
X_val_ischaemia, X_test_ischaemia, y_val_ischaemia, y_test_ischaemia = train_test_split(
    X_test_ischaemia, y_test_ischaemia, test_size=0.25, random_state=42)  # 0.25 * 0.3 = 0.075

# Split the Infection dataset
X_train_infection, X_test_infection, y_train_infection, y_test_infection = train_test_split(
    images_infection, target_labels_infection, test_size=0.3, random_state=42)
X_val_infection, X_test_infection, y_val_infection, y_test_infection = train_test_split(
    X_test_infection, y_test_infection, test_size=0.25, random_state=42)  # 0.25 * 0.3 = 0.075

print("Ischaemia Training set shape:", X_train_ischaemia.shape, y_train_ischaemia.shape)
print("Ischaemia Validation set shape:", X_val_ischaemia.shape, y_val_ischaemia.shape)
print("Ischaemia Test set shape:", X_test_ischaemia.shape, y_test_ischaemia.shape)
print("Infection Training set shape:", X_train_infection.shape, y_train_infection.shape)
print("Infection Validation set shape:", X_val_infection.shape, y_val_infection.shape)
print("Infection Test set shape:", X_test_infection.shape, y_test_infection.shape)

Ischaemia Class Mapping:
non-ischemia: 1
ischemia: 0
Infection Class Mapping:
non-infection: 1
infection: 0
Shape of Ischaemia images array: (9870, 224, 224, 3)
Shape of Ischaemia target labels array: (9870,)
Shape of Infection images array: (5890, 224, 224, 3)
Shape of Infection target labels array: (5890,)
Ischaemia Training set shape: (6909, 224, 224, 3) (6909,)
Ischaemia Validation set shape: (2220, 224, 224, 3) (2220,)
Ischaemia Test set shape: (741, 224, 224, 3) (741,)
Infection Training set shape: (4123, 224, 224, 3) (4123,)
Infection Validation set shape: (1325, 224, 224, 3) (1325,)
Infection Test set shape: (442, 224, 224, 3) (442,)


In [8]:
############################################################################
# ELEPHANT HERDING OPTIMIZATION (EHO) Algorithm Implementation
############################################################################

def initial_variables(size, min_values, max_values, target_function, start_init=None):
    dim = len(min_values)
    
    if start_init is not None:
        start_init = np.atleast_2d(start_init)
        n_rows = size - start_init.shape[0]
        if n_rows > 0:
            rows = np.random.uniform(min_values, max_values, (n_rows, dim))
            start_init = np.vstack((start_init[:, :dim], rows))
        else:
            start_init = start_init[:size, :dim]
        
        fitness_values = np.array([target_function(ind) for ind in start_init])
        # Ensure fitness_values is a 2D array for concatenation
        if fitness_values.ndim == 1:
            fitness_values = fitness_values.reshape(-1, 1)
        print(f"Shape of start_init: {start_init.shape}")
        print(f"Shape of fitness_values: {fitness_values.shape}")
        population = np.hstack((start_init, fitness_values))
    else:
        population = np.random.uniform(min_values, max_values, (size, dim))
        fitness_values = np.array([target_function(ind) for ind in population])
        # Ensure fitness_values is a 2D array for concatenation
        if fitness_values.ndim == 1:
            fitness_values = fitness_values.reshape(-1, 1)
        print(f"Shape of population: {population.shape}")
        print(f"Shape of fitness_values: {fitness_values.shape}")
        population = np.hstack((population, fitness_values))
    
    return population



def update_herd(population, alpha, beta, best_elephant, idx_b, idx_w, min_values, max_values, target_function):
    old_population = np.copy(population)
    cut = population.shape[0]
    dim = len(min_values)
    for i in range(0, cut):
        if i != idx_b and i != idx_w:
            r = np.random.rand(dim)
            population[i, :-1] = np.clip(old_population[i, :-1] + alpha * (best_elephant[:-1] - old_population[i, :-1]) * r, min_values, max_values)
        elif i == idx_b:
            center = np.mean(old_population[:, :-1], axis=0)
            population[i, :-1] = np.clip(beta * center, min_values, max_values)
        elif i == idx_w:
            random_values = np.random.rand(dim)
            population[i, :-1] = np.clip(min_values + (max_values - min_values) * random_values, min_values, max_values)
    
    # Calculate fitness values
    fitness_values = np.array([target_function(ind) for ind in population[:, :-1]])
    population[:, -1] = fitness_values
    
    idx_b = np.argmin(population[:, -1])
    idx_w = np.argmax(population[:, -1])
    if population[idx_b, -1] < best_elephant[-1]:
        best_elephant = np.copy(population[idx_b, :])
    return population, best_elephant, idx_b, idx_w


# Function: EHO
def elephant_herding_optimization(size=50, alpha=0.5, beta=0.1, min_values=[0.0001], max_values=[0.01], generations=5000, target_function=target_function, verbose=True, start_init=None, target_value=None):
    population = initial_variables(size, min_values, max_values, target_function, start_init)
    idx_b = np.argmin(population[:, -1])
    idx_w = np.argmax(population[:, -1])
    best_elephant = population[idx_b, :-1]
    min_values = np.array(min_values)
    max_values = np.array(max_values)
    count = 0
    while count <= generations:
        if verbose:    
            print('Generation: ', count, ' f(x) = ', best_elephant[-1])
        population, best_elephant, idx_b, idx_w = update_herd(population, alpha, beta, best_elephant, idx_b, idx_w, min_values, max_values, target_function)
        if target_value is not None:
            if best_elephant[-1] <= target_value:
                count = 2 * generations
            else:
                count = count + 1
        else:
            count = count + 1
    return best_elephant

############################################################################

In [11]:
import numpy as np
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.layers import Dropout, TimeDistributed, Flatten, LSTM, Dense, BatchNormalization
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tensorflow import keras
import os

# Suppress most TensorFlow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Disable layout optimization to prevent layout warnings
tf.config.optimizer.set_experimental_options({'layout_optimizer': False})

# Define the CNN-LSTM model with specific layers set as trainable
def build_cnn_lstm_model(hp):
    base_model = EfficientNetB7(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
    
    # Freeze all layers except the last 10
    for layer in base_model.layers[:-10]:
        layer.trainable = False

    model = Sequential([
        base_model,
        TimeDistributed(Flatten()),  # Flatten the output for LSTM input
        LSTM(64, dropout=0.3, return_sequences=True),  # First LSTM layer
        LSTM(32, dropout=0.2, return_sequences=True),  # Second LSTM layer
        LSTM(16, dropout=0.2, return_sequences=False),
        Dense(128, activation='relu'),
        Dropout(0.4),
        BatchNormalization(),
        Dense(64, activation='relu'),
        Dropout(0.3),
        BatchNormalization(),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(2, activation='softmax')  # Assuming binary classification
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp['learning_rate']),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# Example hyperparameter tuning setup for learning rate and batch size
def target_function(hyperparams):
    learning_rate = hyperparams[0]
    batch_size = int(hyperparams[1])
    
    hp = {'learning_rate': float(learning_rate)}
    model = build_cnn_lstm_model(hp)
    
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6)
    
    # Train the model with the specified batch size
    history = model.fit(
        X_train_infection, y_train_infection,
        batch_size=batch_size,
        epochs=10,
        validation_data=(X_val_infection, y_val_infection),
        callbacks=[early_stopping, reduce_lr],
        verbose=0
    )
    
    # Return validation loss as the target function metric
    val_loss = min(history.history['val_loss'])
    return val_loss

# Define the new ranges for learning rate and batch size
min_values = [0.0001, 8]  # min learning rate and batch size
max_values = [0.01, 128]  # max learning rate and batch size

# Use the EHO algorithm or other tuning methods to explore these hyperparameters
population = initial_variables(3, min_values, max_values, target_function)
print(population)


W0000 00:00:1729855743.380028     919 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729855743.381033     919 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729855743.381859     919 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729855743.382628     919 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729855743.383514     919 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729855743.384543     919 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729855743.385726     919 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729855743.386865     919 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729855743.387591     919 gp

Shape of population: (3, 2)
Shape of fitness_values: (3, 1)
[[4.55640477e-03 8.85711411e+01 4.52882707e-01]
 [6.55878176e-03 5.83200273e+01 4.99483079e-01]
 [3.94901656e-03 8.09740311e+01 4.66750115e-01]]


In [12]:
# Run Elephant Herding Optimization (EHO)
best_params = elephant_herding_optimization(
    size=2,  # Number of elephants
    alpha=0.2, beta=0.1, 
    min_values=min_values, max_values=max_values,
    generations=2,  # Number of generations
    target_function=target_function 
)

# Print the best found hyperparameters
print("Best Hyperparameters: ", best_params)

W0000 00:00:1729856491.259106     897 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729856491.260208     897 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729856491.261155     897 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729856491.262053     897 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729856491.263102     897 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729856491.264318     897 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729856491.265739     897 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729856491.267111     897 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729856491.267955     897 gp

Shape of population: (2, 2)
Shape of fitness_values: (2, 1)
Generation:  0  f(x) =  110.41186484182302


W0000 00:00:1729856984.658371     908 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729856984.658851     908 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729856984.659124     908 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729856984.659443     908 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729856984.659749     908 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729856984.660053     908 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729856984.660424     908 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729856984.660744     908 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729856984.661116     908 gp

Generation:  1  f(x) =  0.42756885290145874


W0000 00:00:1729858097.236074     900 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729858097.236961     900 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729858097.237660     900 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729858097.238317     900 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729858097.239046     900 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729858097.239883     900 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729858097.240940     900 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729858097.241871     900 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729858097.242503     900 gp

Generation:  2  f(x) =  0.42756885290145874


W0000 00:00:1729858345.744487     900 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729858345.745773     900 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729858345.746778     900 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729858345.747692     900 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729858345.748780     900 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729858345.750026     900 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729858345.751479     900 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729858345.752889     900 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729858345.753757     900 gp

Best Hyperparameters:  [4.65660044e-04 8.00000000e+00 4.27568853e-01]


In [15]:
import time
import numpy as np
from tensorflow import keras

# Extract the best hyperparameters from the EHO result
best_learning_rate = best_params[0]  # First parameter: learning rate
best_batch_size = int(best_params[1])  # Second parameter: batch size (convert to int)

# Train the CNN model using the best hyperparameters
hp_best = {'learning_rate': best_learning_rate}
model = build_cnn_lstm_model(hp_best)

# Set up callbacks for early stopping and learning rate reduction
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6)

# Measure training time
start_train_time = time.time()

# Train the model with the best hyperparameters
history = model.fit(
    X_train_infection, y_train_infection,
    epochs=20,  # You can adjust the number of epochs as needed
    batch_size=best_batch_size,
    validation_data=(X_val_infection, y_val_infection),
    callbacks=[early_stopping, reduce_lr],
    verbose=1  # Set to 1 to show training progress
)

end_train_time = time.time()  # End time for training

# Get the total training time
training_time = end_train_time - start_train_time
print(f"Total Training Time: {training_time:.2f} seconds")

# Get the metrics from the training process
train_acc = history.history['accuracy'][-1]
train_loss = history.history['loss'][-1]
val_acc = history.history['val_accuracy'][-1]
val_loss = history.history['val_loss'][-1]

# Print the training results
print("Training Accuracy: ", train_acc)
print("Training Loss: ", train_loss)
print("Validation Accuracy: ", val_acc)
print("Validation Loss: ", val_loss)

# Measure testing time
start_test_time = time.time()

# Evaluate the model on the test set (assuming you have X_test_infection and y_test_infection)
test_loss, test_acc = model.evaluate(X_test_infection, y_test_infection, verbose=1)

end_test_time = time.time()  # End time for testing

# Get the total testing time
testing_time = end_test_time - start_test_time
print(f"Total Testing Time: {testing_time:.2f} seconds")

# Print the test results
print("Test Accuracy: ", test_acc)
print("Test Loss: ", test_loss)


Epoch 1/20
516/516 ━━━━━━━━━━━━━━━━━━━━ 75s 92ms/step - accuracy: 0.5402 - loss: 0.8138 - val_accuracy: 0.6204 - val_loss: 0.6514 - learning_rate: 4.6566e-04
Epoch 2/20
516/516 ━━━━━━━━━━━━━━━━━━━━ 41s 79ms/step - accuracy: 0.6216 - loss: 0.6688 - val_accuracy: 0.7147 - val_loss: 0.5725 - learning_rate: 4.6566e-04
Epoch 3/20
516/516 ━━━━━━━━━━━━━━━━━━━━ 40s 78ms/step - accuracy: 0.6708 - loss: 0.6223 - val_accuracy: 0.7283 - val_loss: 0.5733 - learning_rate: 4.6566e-04
Epoch 4/20
516/516 ━━━━━━━━━━━━━━━━━━━━ 43s 83ms/step - accuracy: 0.7217 - loss: 0.5851 - val_accuracy: 0.7683 - val_loss: 0.5075 - learning_rate: 4.6566e-04
Epoch 5/20
516/516 ━━━━━━━━━━━━━━━━━━━━ 42s 82ms/step - accuracy: 0.7265 - loss: 0.5613 - val_accuracy: 0.7502 - val_loss: 0.5169 - learning_rate: 4.6566e-04
Epoch 6/20
516/516 ━━━━━━━━━━━━━━━━━━━━ 42s 81ms/step - accuracy: 0.7522 - loss: 0.5415 - val_accuracy: 0.7819 - val_loss: 0.4897 - learning_rate: 4.6566e-04
Epoch 7/20
516/516 ━━━━━━━━━━━━━━━━━━━━ 42s 81ms/ste

W0000 00:00:1729859957.618085     946 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729859957.618854     946 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729859957.619353     946 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729859957.619827     946 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729859957.620301     946 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729859957.620781     946 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729859957.621338     946 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729859957.621895     946 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729859957.622526     946 gp

14/14 ━━━━━━━━━━━━━━━━━━━━ 2s 108ms/step - accuracy: 0.8651 - loss: 0.3688
Total Testing Time: 1.75 seconds
Test Accuracy:  0.8597285151481628
Test Loss:  0.38820788264274597


W0000 00:00:1729859959.041940     898 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729859959.042549     898 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729859959.043009     898 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729859959.043447     898 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729859959.043910     898 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729859959.044386     898 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729859959.044931     898 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729859959.045483     898 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1729859959.046078     898 gp